In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
sample_submission = pd.read_csv("/kaggle/input/tabular-playground-series-oct-2021/sample_submission.csv")
train = pd.read_csv("/kaggle/input/tabular-playground-series-oct-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-oct-2021/test.csv")

In [3]:
sample_submission.head()

In [4]:
train.head()

In [5]:
test.head()

In [6]:
train_null_cols = [col for col in train.iloc[ : , 1 : ].columns if train[col].isnull().sum() != 0]
train_null_cols

In [7]:
test_null_cols = [col for col in test.iloc[ : , 1 : ].columns if test[col].isnull().sum() != 0]
test_null_cols

In [8]:
train.info()

In [9]:
train_int_cols = [col for col in train.iloc[ : , 1 : ].columns if train[col].dtype == "int64"]
len(train_int_cols)

In [10]:
train_float_cols = [col for col in train.iloc[ : , 1 : ].columns if train[col].dtype == "float64"]
len(train_float_cols)

In [11]:
test.info()

In [12]:
test_int_cols = [col for col in test.iloc[ : , 1 : ].columns if test[col].dtype == "int64"]
len(test_int_cols)

In [13]:
test_float_cols = [col for col in test.iloc[ : , 1 : ].columns if test[col].dtype == "float64"]
len(test_float_cols)

In [14]:
#train_int_unique = [train[col].unique() for col in train[train_int_cols].columns]
#train_int_unique

train_cols_binary = [col for col in train[train_int_cols].columns if np.all(train[col].unique() == [0, 1]) | np.all(train[col].unique() == [1, 0])]
len(train_cols_binary)

In [15]:
test_cols_binary = [col for col in test[test_int_cols].columns if np.all(test[col].unique() == [0, 1]) | np.all(test[col].unique() == [1, 0])]
len(test_cols_binary)

In [25]:
from sklearn.model_selection import train_test_split

y_train = train['target']
X_train = train.drop('target', axis = 1)
X_test = test

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.3,
                                                      random_state =0, stratify = y_train)

In [27]:
import lightgbm as lgb

lgb_train = lgb.Dataset(X_train, y_train, categorical_feature = train_cols_binary[ : -1])
lgb_eval = lgb.Dataset(X_valid, y_valid, categorical_feature = test_cols_binary)

params = {
    'objective': 'binary'
}

model = lgb.train(params, lgb_train,
                  valid_sets = [lgb_train, lgb_eval],
                  verbose_eval = 10,
                  num_boost_round = 1000,
                  early_stopping_rounds = 10)

y_pred = model.predict(X_test, num_iteration = model.best_iteration)